<a href="https://colab.research.google.com/github/qkrjuyeol/multi-cctv/blob/main/%EC%9D%B4%EC%83%81%ED%96%89%EB%8F%99_%ED%83%90%EC%A7%80_%EB%AA%A8%EB%8D%B8_%ED%85%8C%EC%8A%A4%ED%8A%B8_%EC%BD%94%EB%93%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import cv2
import torch
import torchvision.models.video as video_models
import torch.nn as nn
import numpy as np

In [ ]:
# ===== 설정 =====
video_path = "test1.mp4"  # 분석할 영상 경로
clip_duration_sec = 2     # 클립 길이
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# ===== 모델 불러오기 =====
model = video_models.r3d_18(weights=None)
model.fc = nn.Linear(model.fc.in_features, 1)
model.load_state_dict(torch.load("dump_detection_model_r3d18.pth", map_location=device))
model = model.to(device)
model.eval()

In [ ]:
# ===== 클립 전처리 함수 =====
def preprocess_clip(frames):
    frames = [cv2.resize(f, (112, 112)) for f in frames]
    frames = [cv2.cvtColor(f, cv2.COLOR_BGR2RGB) for f in frames]
    arr = np.stack(frames).transpose(3, 0, 1, 2)  # (C, T, H, W)
    tensor = torch.tensor(arr, dtype=torch.float32) / 255.0
    return tensor.unsqueeze(0).to(device)

In [ ]:
# ===== 영상 열기 =====
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    raise RuntimeError("영상 열기 실패")

fps = cap.get(cv2.CAP_PROP_FPS)
clip_len = int(fps * clip_duration_sec)
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

print(f"FPS: {fps:.2f}, 총 프레임 수: {total_frames}, 클립 길이: {clip_len}프레임")

drop_ranges = []

FPS: 29.97, 총 프레임 수: 9059, 클립 길이: 59프레임


In [ ]:
# ===== 클립 추출 및 예측 =====
for i in range(0, total_frames - clip_len, clip_len):
    cap.set(cv2.CAP_PROP_POS_FRAMES, i)
    frames = []
    for _ in range(clip_len):
        ret, frame = cap.read()
        if not ret:
            break
        frames.append(frame)

    if len(frames) == clip_len:
        input_tensor = preprocess_clip(frames)
        with torch.no_grad():
            out = model(input_tensor)
            prob = torch.sigmoid(out).item()

        start_sec = int(i // fps)
        end_sec = int((i + clip_len) // fps)

        print(f"[{start_sec}s ~ {end_sec}s] drop 확률: {prob:.4f}")

        if prob > 0.5:
            drop_ranges.append((start_sec, end_sec))

cap.release()

[0s ~ 1s] drop 확률: 0.0013
[1s ~ 3s] drop 확률: 0.0013
[3s ~ 5s] drop 확률: 0.0019
[5s ~ 7s] drop 확률: 0.0026
[7s ~ 9s] drop 확률: 0.0034
[9s ~ 11s] drop 확률: 0.0040
[11s ~ 13s] drop 확률: 0.0041
[13s ~ 15s] drop 확률: 0.0048
[15s ~ 17s] drop 확률: 0.0054
[17s ~ 19s] drop 확률: 0.0052
[19s ~ 21s] drop 확률: 0.0037
[21s ~ 23s] drop 확률: 0.0045
[23s ~ 25s] drop 확률: 0.0017
[25s ~ 27s] drop 확률: 0.0019
[27s ~ 29s] drop 확률: 0.0014
[29s ~ 31s] drop 확률: 0.0012
[31s ~ 33s] drop 확률: 0.0017
[33s ~ 35s] drop 확률: 0.0020
[35s ~ 37s] drop 확률: 0.0018
[37s ~ 39s] drop 확률: 0.0022
[39s ~ 41s] drop 확률: 0.0037
[41s ~ 43s] drop 확률: 0.0059
[43s ~ 45s] drop 확률: 0.0319
[45s ~ 47s] drop 확률: 0.0301
[47s ~ 49s] drop 확률: 0.0170
[49s ~ 51s] drop 확률: 0.0246
[51s ~ 53s] drop 확률: 0.0369
[53s ~ 55s] drop 확률: 0.0497
[55s ~ 57s] drop 확률: 0.0491
[57s ~ 59s] drop 확률: 0.0289
[59s ~ 61s] drop 확률: 0.1750
[61s ~ 62s] drop 확률: 0.2079
[62s ~ 64s] drop 확률: 0.0963
[64s ~ 66s] drop 확률: 0.2028
[66s ~ 68s] drop 확률: 0.1804
[68s ~ 70s] drop 확률: 0.2018
[70

In [ ]:
# ===== 결과 출력 =====
print("\nDrop 행동 감지 시간 범위 (초 단위):")
if drop_ranges:
    for s, e in drop_ranges:
        print(f"{s}s ~ {e}s")
else:
    print("감지된 drop 행동이 없습니다.")


Drop 행동 감지 시간 범위 (초 단위):
269s ~ 271s
